In [ ]:
%%writefile fibonacci.py
import sys

def fib(n):
    a, b = 0, 1
    for _ in range(n):
        a, b = b, a + b
    return a

if __name__ == "__main__":
    n = int(sys.argv[1])
    result = fib(n)
    print(result)


In [ ]:
%%writefile factorial.py
import sys

def fact(n):
    if n > 1:
        return n * fact(n - 1)
    return 1

if __name__ == "__main__":
    n = int(sys.argv[1])
    result = fact(n)
    print(result)


In [ ]:
%%writefile plot.py
import matplotlib.pyplot as plt
import sys

fib_x = int(sys.argv[1])
fact_x = int(sys.argv[2])

plt.figure(figsize=(10, 6))
plt.bar(["fib(x)", "fact(x)"], [fib_x, fact_x], log=True)
plt.yscale("log")
plt.text(0, fib_x, str(fib_x), ha='center', va='bottom')
plt.text(1, fact_x, str(fact_x), ha='center', va='bottom')
plt.savefig("plot.png")
plt.close()


In [ ]:
from Pegasus.api import *

# properties here
props = Properties()
props["pegasus.mode"] = "development"

sc = SiteCatalog()
local = Site("local", arch=Arch.X86_64, os_type=OS.LINUX)
local.add_directories(Directory(Directory.SHARED_SCRATCH, path="/tmp/scratch"))
sc.add_sites(local)

# Define the workflow
wf = Workflow("math_operations")

# jobs
job_fib = Job("fibonacci.py")
job_fib.add_args("35")
job_fib.set_stdout("fibonacci_output.txt")

job_fact = Job("factorial.py")
job_fact.add_args("35")
job_fact.set_stdout("factorial_output.txt")

job_plot = Job("plot.py")
job_plot.add_args("$(job_fib.stdout)", "$(job_fact.stdout)")
job_plot.set_stdout("plot.png")


wf.add_jobs(job_fib, job_fact, job_plot)
wf.add_dependency(job_fib, children=[job_plot])
wf.add_dependency(job_fact, children=[job_plot])

# yml file 
wf.write(file="math_operations.yml")

# Run the workflow
!pegasus-plan --conf pegasus.properties --sites local --dir work --dax math_operations.yml --output-site local --submit
